In [1]:
from pathlib import Path
from YOLOv8BeyondEarth.predict import get_sliced_prediction
from rastertools_BOULDERING import raster, metadata as raster_metadata, crs as raster_crs
from shptools_BOULDERING import shp, geometry as shp_geom, geomorph as shp_geomorph, annotations as shp_anno, metrics as shp_metrics
from sahi import AutoDetectionModel

## Sliced prediction (get_sliced_prediction)

Function to slice a large image and get prediction for each slice + combine all of the predictions back to the full image.

The time to run the script is dependent on the number of predictions over the whole image. This is because the algorithm loops through each prediction in each slice and transform the bool_mask to polygon. 

**Few notes:**

1. YOLOv8 model expects the image to be in BGR with the following shape (height, width, 3). 
2. If you want to detect very small objects, the slice size should be
   decreased, and the inference size should be increased.
   - slice_size = 256
   - inference_size = 1024.
3. If you are in the opposite situation, where you realized that most of the large boulders are missed out. You can
   increase the slice height and width.
   - slice_size = 1024
   - inference_size  = 512, 1024.

You can get the best of both worlds by combining predictions (2) and (3) with Non-Maximum Suppresion (NMS). Obviously the larger the slice size, and the larger the inference size, the more time it takes to run this script. 

Test Time Augmentation could be included too, but it takes lot of time to run it.. 

Note that the bboxes (in absolute coordinates) are calculated from the bounds of the polygons after the
predictions are computed.

**Example:**

Let's have a look at an example. We want to automatically detect boulders on NAC image M139694087LE, which depicts about half of the Censorinus impact crater on the lunar surface. Censorinus is relatively fresh impact crater with a diameter of about 4 km. More than 250,000 boulders are located close to its vicinity (Krishna et al., 2016). I have provided the NAC image on my GoogleDrive so that you do not have to process it. edictions are computed.redictions are computed.n. 

In [2]:
home_p = Path.home()
work_dir= home_p / "tmp" / "YOLOv8BeyondEarth"
raster_dir = (work_dir / "raster")
model_dir = (work_dir / "yolov8_model")

# Let's define the temporary working directories (feel free to change where the raster is saved to)
work_dir.mkdir(parents=True, exist_ok=True)
raster_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True) 

In [3]:
raster_dir

WindowsPath('C:/Users/nilscp/tmp/YOLOv8BeyondEarth/raster')

In [17]:
url_raster = "https://drive.google.com/uc?id=1o9A0GSHQ0m_XTPAgDRDTD25xlgWutqMW"
url_yolov8_model_weights = "https://drive.google.com/uc?id=1DJ3Ek4NI1uEzlB1pyor-KDRN8_pEorVp"

You need to `pip install gdown` if needed

In [11]:
import gdown

#### Downloading of example image and model weights

In [20]:
gdown.download(url_raster, (raster_dir / "M139694087LE.tif").as_posix(), quiet=True)
gdown.download(url_yolov8_model_weights, (model_dir / "yolov8-m-boulder-detection-tmp.pt").as_posix(), quiet=True)

'C:/Users/nilscp/tmp/YOLOv8BeyondEarth/yolov8_model/yolov8-m-boulder-detection-tmp.pt'

In [4]:
in_raster = raster_dir / "M139694087LE.tif"
model_weights = model_dir / "yolov8-m-boulder-detection-tmp.pt"

### Boulder detection
#### Loading of the model

In [7]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=model_weights.as_posix(),
    confidence_threshold=0.1, # this parameter will be changed by the function
    device="cuda:0",  # or cpu, please run this code on a computer with a graphical card, otherwise it will take a long time! 
    image_size=1024) # this parameter will be changed by the function

#### Predictions over the whole NAC image

In order to detect boulders of different sizes, we are running the code for 4 different slice sizes (256x256, 512x512, 768x768 and 1024x1024). The sliced images are then upsampled to 1024x1024 (the inference size). We here used a 20% overlap between the sliced images to avoid for edge artifacts. A NMS threshold is set to 0.20 to filter away overlapping bouding boxes/masks (this step is based on the amount of overlap between bounding boxes). Only boulders with more than 6 pixels are kept. The `downscale_pred` flag controls if the polygons are derived from the masks generated during the inference (1024x1024 in this case), or if they are computed from the resized masks (resized from inference to slice size). Using this flag speed up significantly the code, and it is almost necessary for the code to run at an OK speed for images with lot of predictions. However, the boulder outlines may suffer from this approximation. You can play a bit around with this flag, and see how it impacts the performances.

In [5]:
output_dir = (work_dir / "inference")
output_dir.mkdir(parents=True, exist_ok=True)

interim_dir = (work_dir / "interim_dir")
interim_dir.mkdir(parents=True, exist_ok=True) 

In [8]:
slice_sizes = [256, 512, 768, 1024] 
confidence_threshold = 0.10
inference_size = 1024
overlap_height_ratio = 0.20

for slice_size in slice_sizes:
        __, __ = get_sliced_prediction(in_raster,
                            detection_model=detection_model,
                            confidence_threshold=confidence_threshold,
                            has_mask=True,
                            output_dir=output_dir,
                            interim_file_name=in_raster.stem,  # YOU CAN OPTIONALLY SAVE SLICES WITH SPECIFIC NAME
                            interim_dir=interim_dir,  # YOU CAN OPTIONALLY SAVE SLICES TO THIS INTERIM DIRECTORY
                            slice_size=slice_size,
                            inference_size=inference_size,
                            overlap_height_ratio=0.2,
                            overlap_width_ratio=0.2,
                            min_area_threshold=6,
                            downscale_pred=True, # if True, the predicted mask is downscaled to the slice_size, decreasing the coputational time.  
                            postprocess= True,
                            postprocess_match_threshold=0.2, # 
                            postprocess_class_agnostic=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [09:32<00:00,  1.32s/it]


It takes about 30 min for a NAC image (if four different slice sizes are run).

### Crater detection

In [9]:
url_yolov8_crater_model_weights = "https://drive.google.com/uc?id=10lcw015kekhwZtDa0u7VuzgqG7NuObhZ"

In [12]:
gdown.download(url_yolov8_crater_model_weights, (model_dir / "yolov8-m-crater-detection-tmp.pt").as_posix(), quiet=True)

'C:/Users/nilscp/tmp/YOLOv8BeyondEarth/yolov8_model/yolov8-m-crater-detection-tmp.pt'

In [13]:
model_weights_crater = model_dir / "yolov8-m-crater-detection-tmp.pt"

In [14]:
crater_detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=model_weights_crater.as_posix(),
    confidence_threshold=0.1, # this parameter will be changed by the function
    device="cuda:0",  # or cpu, please run this code on a computer with a graphical card, otherwise it will take a long time! 
    image_size=1024) # this parameter will be changed by the function

In [15]:
output_dir = (work_dir / "inference_crater")
output_dir.mkdir(parents=True, exist_ok=True)

In [17]:
slice_sizes = [256] 
confidence_threshold = 0.10
inference_size = 512
overlap_height_ratio = 0.20

for slice_size in slice_sizes:
        __, __ = get_sliced_prediction(in_raster,
                            detection_model=crater_detection_model,
                            confidence_threshold=confidence_threshold,
                            has_mask=True,
                            output_dir=output_dir,
                            interim_file_name=None,  # YOU CAN OPTIONALLY SAVE SLICES WITH SPECIFIC NAME
                            interim_dir=None,  # YOU CAN OPTIONALLY SAVE SLICES TO THIS INTERIM DIRECTORY
                            slice_size=slice_size,
                            inference_size=inference_size,
                            overlap_height_ratio=0.2,
                            overlap_width_ratio=0.2,
                            min_area_threshold=6,
                            downscale_pred=True, # if True, the predicted mask is downscaled to the slice_size, decreasing the coputational time.  
                            postprocess= True,
                            postprocess_match_threshold=0.2, # 
                            postprocess_class_agnostic=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 7225/7225 [02:37<00:00, 46.01it/s]
